In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns


# Specify the path to the CSV files
csv_path = 'historical_data/Battles_WW1.csv'

# Load the CSV files into a GeoPandas DataFrame
df = gpd.read_file(csv_path)

# Convert the coordinates column to geometry
df['geometry'] = df['Coordinates'].apply(lambda coord: Point(float(coord.split(',')[1]), float(coord.split(',')[0])))


# convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])
df['EndDate'] = pd.to_datetime(df['EndDate'])
# Sort the DataFrame by the date column
df = df.sort_values('Date')




# Create a timeline graph
fig, ax = plt.subplots(figsize=(10, 6))

#sns.lineplot(df['Date'], range(len(df)), marker='o', linestyle='-', color='b', ax=ax)

ax.plot(df['Date'], range(len(df)), marker='o', linestyle='-', color='b')

# Add labels to the graph
for i, row in df.iterrows():
    ax.text(row['Date'], i, row['Battle'], ha='right', va='center')

# Set the x-axis label
ax.set_xlabel('Date')

# Set the y-axis label
ax.set_ylabel('Battle')

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Show the graph
plt.show()






In [ ]:
import plotly.figure_factory as ff

# Preparing data for Gantt chart
# Ensure your dataframe has columns 'Battle', 'Start', and 'End'
df_gantt = df[['Battle', 'Date', 'EndDate','Front']]

# if the EndDate is the same as the StartDate, add one day to the EndDate
#df_gantt['EndDate'] = df_gantt['EndDate'].apply(lambda x: x + pd.DateOffset(days=1) if x == df_gantt['Date'] else x)

df_gantt['EndDate'] = df.apply(lambda row: row['EndDate'] + pd.Timedelta(days=1) 
                         if row['Date'] == row['EndDate'] else row['EndDate'], axis=1)

df_gantt['Task'] = df_gantt['Battle']
df_gantt['Start'] = df_gantt['Date']
df_gantt['Finish'] = df_gantt['EndDate']
df_gantt['Resource'] = df_gantt['Front']  # or 'Eastern Front' based on your data

# Creating Gantt chart
fig = ff.create_gantt(df_gantt, index_col='Resource', show_colorbar=True, group_tasks=True)

# Showing the figure
fig.show()
#df_gantt

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df['DateName']= df['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Battle']

# Create Map Plot
map_trace = go.Scattergeo(
    lon = df.geometry.x,
    lat = df.geometry.y,
    text = df['DateName'],
    mode = 'markers',
    marker = dict(size = 8, color = 'red')
)

map_layout = go.Layout(
    title = 'World War I Battles',
    geo = dict(showland = True)
)

map_fig = go.Figure(data = [map_trace], layout = map_layout)
# Display the figures
map_fig.show()

